# Piecewise Conversion

Model equipment with **load-dependent efficiency** using piecewise linear approximation.

**User Story:** A gas engine's efficiency varies with load - lower at part-load, optimal at mid-load. We want to capture this non-linear behavior in our optimization.

In [1]:
import numpy as np
import pandas as pd

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## The Problem

Real equipment efficiency varies with operating point:

| Load Level | Electrical Efficiency | Reason |
|------------|----------------------|--------|
| 25-50% (part load) | 32-38% | Throttling losses |
| 50-75% (mid load) | 38-42% | Near design point |
| 75-100% (full load) | 42-40% | Thermal limits |

A constant efficiency assumption misses this behavior.

## Define the Efficiency Curve

Each `Piece` defines corresponding fuel input and electricity output ranges:

In [2]:
piecewise_efficiency = fx.PiecewiseConversion(
    {
        'Fuel': fx.Piecewise(
            [
                fx.Piece(start=78, end=132),  # Part load
                fx.Piece(start=132, end=179),  # Mid load
                fx.Piece(start=179, end=250),  # Full load
            ]
        ),
        'Elec': fx.Piecewise(
            [
                fx.Piece(start=25, end=50),  # 32% -> 38% efficiency
                fx.Piece(start=50, end=75),  # 38% -> 42% efficiency
                fx.Piece(start=75, end=100),  # 42% -> 40% efficiency
            ]
        ),
    }
)

## Build and Solve

In [3]:
timesteps = pd.date_range('2024-01-22', periods=24, freq='h')

# Demand varies through the day (30-90 kW, within piecewise range 25-100)
elec_demand = 60 + 30 * np.sin(np.arange(24) * np.pi / 12)

fs = fx.FlowSystem(timesteps)
fs.add_elements(
    fx.Bus('Gas'),
    fx.Bus('Electricity'),
    fx.Effect('costs', '€', is_standard=True, is_objective=True),
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=300, effects_per_flow_hour=0.05)]),
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas')],
        outputs=[fx.Flow('Elec', bus='Electricity')],
        piecewise_conversion=piecewise_efficiency,
    ),
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=elec_demand)]),
)

fs.optimize(fx.solvers.HighsSolver());

## Visualize the Efficiency Curve

In [4]:
fs.components['GasEngine'].piecewise_conversion.plot(x_flow='Fuel')

PlotResult(data=<xarray.Dataset> Size: 268B
Dimensions:  (trace: 3, point: 2)
Coordinates:
  * point    (point) <U5 40B 'start' 'end'
Dimensions without coordinates: trace
Data variables:
    Fuel     (trace, point) int64 48B 78 132 132 179 179 250
    output   (trace, point) int64 48B 25 50 50 75 75 100
    flow     (trace) <U4 48B 'Elec' 'Elec' 'Elec'
    piece    (trace) <U7 84B 'Piece 0' 'Piece 1' 'Piece 2', figure=Figure({
    'data': [{'hovertemplate': 'piece=Piece 0<br>line_id=0<br>Fuel=%{x}<br>output=%{y}<extra></extra>',
              'legendgroup': 'Piece 0',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Piece 0',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'TgCEAA==', 'dtype': 'i2'},
              'xaxis': 'x',
              'y': {'bdata': 'GTI=', 'dtype': 'i1'},
              'yaxis': 'y'},
             {'hovertemplate': 'piece=Piece 1<br>line_id=1<br>Fuel=%{x}<br>output=%{y}<extra></extra>',
              'legendgroup': 'Piece 1',
              'line': {'color': '#636EFA', 'dash': 'dot'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Piece 1',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'hACzAA==', 'dtype': 'i2'},
              'xaxis': 'x',
              'y': {'bdata': 'Mks=', 'dtype': 'i1'},
              'yaxis': 'y'},
             {'hovertemplate': 'piece=Piece 2<br>line_id=2<br>Fuel=%{x}<br>output=%{y}<extra></extra>',
              'legendgroup': 'Piece 2',
              'line': {'color': '#636EFA', 'dash': 'dash'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Piece 2',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'swD6AA==', 'dtype': 'i2'},
              'xaxis': 'x',
              'y': {'bdata': 'S2Q=', 'dtype': 'i1'},
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'piece'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Piecewise Conversion'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Fuel'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': ''}}}
}))

## Results

In [5]:
fs.statistics.plot.balance('Electricity')

PlotResult(data=<xarray.Dataset> Size: 600B
Dimensions:          (time: 25)
Coordinates:
  * time             (time) datetime64[ns] 200B 2024-01-22 ... 2024-01-23
Data variables:
    GasEngine(Elec)  (time) float64 200B -60.0 -67.76 -75.0 ... -45.0 -52.24 nan
    Load(Elec)       (time) float64 200B 60.0 67.76 75.0 ... 45.0 52.24 nan, figure=Figure({
    'data': [{'fillcolor': '#EF553B',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f}<extra></extra>',
              'legendgroup': 'GasEngine(Elec)',
              'line': {'color': '#EF553B', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'GasEngine(Elec)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-01-22T00:00:00.000000000', '2024-01-22T01:00:00.000000000',
                          '2024-01-22T02:00:00.000000000', '2024-01-22T03:00:00.000000000',
                          '2024-01-22T04:00:00.000000000', '2024-01-22T05:00:00.000000000',
                          '2024-01-22T06:00:00.000000000', '2024-01-22T07:00:00.000000000',
                          '2024-01-22T08:00:00.000000000', '2024-01-22T09:00:00.000000000',
                          '2024-01-22T10:00:00.000000000', '2024-01-22T11:00:00.000000000',
                          '2024-01-22T12:00:00.000000000', '2024-01-22T13:00:00.000000000',
                          '2024-01-22T14:00:00.000000000', '2024-01-22T15:00:00.000000000',
                          '2024-01-22T16:00:00.000000000', '2024-01-22T17:00:00.000000000',
                          '2024-01-22T18:00:00.000000000', '2024-01-22T19:00:00.000000000',
                          '2024-01-22T20:00:00.000000000', '2024-01-22T21:00:00.000000000',
                          '2024-01-22T22:00:00.000000000', '2024-01-22T23:00:00.000000000',
                          '2024-01-23T00:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAATsAVHjS77vBQwAAAAAAAwF' ... 'AAAIBGwNfDl4kiHkrAAAAAAAAA+P8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'fillcolor': '#00CC96',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f}<extra></extra>',
              'legendgroup': 'Load(Elec)',
              'line': {'color': '#00CC96', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Load(Elec)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'positive',
              'type': 'scatter',
              'x': array(['2024-01-22T00:00:00.000000000', '2024-01-22T01:00:00.000000000',
                          '2024-01-22T02:00:00.000000000', '2024-01-22T03:00:00.000000000',
                          '2024-01-22T04:00:00.000000000', '2024-01-22T05:00:00.000000000',
                          '2024-01-22T06:00:00.000000000', '2024-01-22T07:00:00.000000000',
                          '2024-01-22T08:00:00.000000000', '2024-01-22T09:00:00.000000000',
                          '2024-01-22T10:00:00.000000000', '2024-01-22T11:00:00.000000000',
                          '2024-01-22T12:00:00.000000000', '2024-01-22T13:00:00.000000000',
                          '2024-01-22T14:00:00.000000000', '2024-01-22T15:00:00.000000000',
                          '2024-01-22T16:00:00.000000000', '2024-01-22T17:00:00.000000000',
                          '2024-01-22T18:00:00.000000000', '2024-01-22T19:00:00.000000000',
                          '2024-01-22T20:00:00.000000000', '2024-01-22T21:00:00.000000000',
                          '2024-01-22T22:00:00.000000000', '2024-01-22T23:00:00.000000000',
                          '2024-01-23T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
# Verify efficiency varies with load
fuel = fs.solution['GasEngine(Fuel)|flow_rate']
elec = fs.solution['GasEngine(Elec)|flow_rate']
efficiency = elec / fuel

print(f'Efficiency range: {float(efficiency.min()):.1%} - {float(efficiency.max()):.1%}')
print(f'Total cost: {fs.solution["costs"].item():.2f} €')

Efficiency range: 33.8% - 41.9%
Total cost: 182.96 €


## Key Points

**Syntax:**
```python
fx.PiecewiseConversion({
    'Input': fx.Piecewise([fx.Piece(start=a, end=b), ...]),
    'Output': fx.Piecewise([fx.Piece(start=x, end=y), ...]),
})
```

**Rules:**
- All flows must have the **same number of segments**
- Segments typically **connect** (end of N = start of N+1)
- Efficiency = output / input at each point

**Time-varying:** Pass arrays instead of scalars to model changing limits (e.g., temperature derating).

**Next:** See [06c-piecewise-effects](06c-piecewise-effects.ipynb) for non-linear investment costs.